## NASA Center Analysis
<details>
<summary>IMPORTANT (FEB 2025)</summary>
<br>
NEX GDDP currently has a bug that is affected tmin (minimum temperature) and tas (average temperature) for some Centers, only for SSP 1-2.6 and 3-7.0. These are the following Centers that are affected:

AMES
LARC
GISS
JPL
JSC
KSC
WFF

That means any variable that includes tmin or tas, for SSP 1-2.6 and 3-7.0, will be affected, and we will plan on not using those variables for now. The variables include:
Max DTR, Tmin >20C, Tmin < 0C, Coldest Tmin of the Year, Annual Average Tmin, any of the humid heat diagnostics (e.g., heat index, WBGT, etc).
</details>

<details>
<summary>VARIABLES</summary>

| Variable Name       | Long Name                                          | Variable Category | Units     | Description                                                                                                                                                          |
| ------------------- | -------------------------------------------------- | ----------------- | --------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| tmax_days_35C       | \# days Tmax ≥35°C                                 | extreme index     | \# days   | Number of days, per year, with Tmax >=35C                                                                                                                            |
| tmax_days_90th      | \# days Tmax ≥90th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 90th percentile. 90th percentile calculated using all daily tmax values from 1995-2014.                                          |
| tmax_days_95th      | \# days Tmax ≥95th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 95th percentile. 95th percentile calculated using all daily tmax values from 1995-2014.                                          |
| tmax_days_99th      | \# days Tmax ≥99th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 99th percentile. 99th percentile calculated using all daily tmax values from 1995-2014.                                          |
| Hottest_Tmax        | Hottest Tmax of the Year (°C)                      | extreme index     | degrees C | Hottest Tmax value every year                                                                                                                                        |
| Max_DTR             | Largest Diurnal Temperature Range of the Year (°C) | extreme index     | degrees C | largest diurnal temperature range (tmax minus tmin) each year                                                                                                        |
| tmin_tropnights_20C | \# days Tmin ≥20°C                                 | extreme index     | \# days   | Number of days, per year with Tmin >=20C                                                                                                                             |
| tmin_frostdays_0C   | \# days Tmin ≤0°C                                  | extreme index     | \# days   | Number of days per year with Tmin <=0C                                                                                                                               |
| Coldest_Tmin        | Coldest Tmin of the Year (°C)                      | extreme index     | degrees C | Coldest minimum temperature each year                                                                                                                                |
| prec_days_dry       | \# days with precipitation ≤0.001 in               | extreme index     | \# days   | Number of days, per year, where precipitation <=1e-3 inches                                                                                                          |
| prec_days_oneinch   | \# days with precipitation ≥1 in                   | extreme index     | \# days   | Number of days, per year, where precipitation >=1 inch                                                                                                               |
| prec_days_90th      | \# days with precipitation ≥90th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=90th percentile. 90th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| prec_days_95th      | \# days with precipitation ≥95th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=95th percentile. 95th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| prec_days_99th      | \# days with precipitation ≥99th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=99th percentile. 99th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| tmax_annave         | Annual Average Tmax (°C)                           | annual average    | degrees C | Annual average maximum daily temperature                                                                                                                             |
| tmin_annave         | Annual Average Tmin (°C)                           | annual average    | degrees C | Annual average minimum daily temperature                                                                                                                             |
| prec_annave         | Annual Total Precipitation (mm)                    | annual SUM        | degrees C | Annual SUM of precipitation                                                                                                                                          |</details>

In [1]:
# Imports
import os
import warnings

import numpy as np
import pandas as pd
import pandasql as psql

# Suppress warnings
warnings.filterwarnings('ignore')

## Initialization

In [2]:
path = 'updated_extremes'  # data directory
center = 'LARC'.upper()    # NASA center to analyze
only_future = True         # flag to use only 2020-2099
ssp = ['ssp126', 'ssp245', 'ssp370'] # scenarios to use

In [3]:
# DO NOT CHANGE THIS CELL
# File name convention: variable_CENTER_ssp###.csv

# NASA Centers
centers = ['AMES', 'GSFC', 'JPL', 'KSC', 'MSFC', 'MAF', 'GISS',
           'LARC', 'SSC', 'GRC', 'WFF', 'JSC', 'WSTF', 'AFRC']

# Check if the provided center is valid
if center not in centers:
    raise ValueError(f'{center} not in {centers}')

# Variable unit: number of DAYS when... assume others are celsius
day_unit = ['days', 'tropnights']

# Shared Socioeconomic Pathways (SSPs) climate scenarios
print(f'Available: {sorted(list({f.split('_')[-1][:-4] for f in os.listdir(path) 
                                 if 'ssp' in f.split('_')[-1]}))}')
# Time periods: 10 years before+after a decade
time_periods = {'short': (2020, 2049),  # 2030's: 2020-2029, 2030-2039, 2040-2049
                'mid':   (2040, 2069),  # 2050's: 2040-2049, 2050-2059, 2060-2069
                'long':  (2070, 2099),  # 2080's: 2070-2079, 2080-2089, 2090-2099
                }

Available: ['ssp126', 'ssp245', 'ssp370', 'ssp585']


# Get Files/Data

In [4]:
def get_files(path: str, center: str):
    '''Returns list of all csv files in the directory that contain the center and ssp name'''
    return [os.path.join(path, f) for f in os.listdir(path) 
             if center in f and any(s in f for s in ssp) and f.endswith('.csv')]

def check_df_consistency(df_list: list[pd.DataFrame]):
    '''Returns T/F if all dataframes in the list have the same column names and index values'''
    if not df_list:
        return False
    
    # Get reference column names and index values from the first dataframe
    ref_cols, ref_index = list(df_list[0].columns), list(df_list[0].index)
    
    # Check if all other dataframes have the same column names and index values
    return all(list(df.columns) == ref_cols and list(df.index) == ref_index 
               for df in df_list[1:])

def label_term(year: int):
    '''Returns list of time period labels for a given year'''
    return [t for t, (s, e) in time_periods.items() if s <= year <= e]


def preprocess(filename: str, only_future: bool=True):
    '''Returns a preprocessed pandas DataFrame from a csv file'''
    df = pd.read_csv(filename)
    
    # Extract variable name from the filename
    name = filename.split('/')[-1][:-4].split('_')
    var = '_'.join(name[:-2])
    
    # Add new columns: term, scenario, and variable
    df.insert(0, 'term', df.years.apply(label_term))
    df.insert(0, 'scenario', name[-1])
    df.insert(0, 'variable', var + ('_d' if any(d in filename for d in day_unit) else '_c'))
    
    # Explode the 'term' column (in case a year belongs to multiple terms)
    df = df.explode('term')

    # Remove rows with NaN terms if only_future is True, otherwise return all rows
    return df.dropna(subset=['term']) if only_future else df # nan's (unlabeled) assumed to be past data

In [5]:
# Preprocess all files as df's
files = sorted(get_files(path, center))
df = [preprocess(f, only_future) for f in files]

# Check if all dataframes have the same format
if not check_df_consistency(df):
    raise ValueError('DataFrames are inconsistent')

# Combine all dataframes into one
df = pd.concat(df).reset_index(drop=True)
cols = ['variable', 'scenario', 'term']

# Check the number of years per time period (expected is 30)
years_per_term = df.groupby(cols).size().unique()
if len(years_per_term) != 1 or years_per_term[0] != 30:
    raise ValueError(
        f'# of years per time period is incorrect: {years_per_term}')

######################################
# REMOVE IF SSP 126, 370 ERROR FIXED
######################################
# For: AMES LARC GISS JPL JSC KSC WFF
# Erronous: Max DTR, Tmin >20C, Tmin < 0C, Coldest Tmin of the Year, Annual Average Tmin, any of the humid heat diagnostics (e.g., heat index, WBGT, etc) in ssp126 and 370
errors = ['Max_DTR', 'tmin', 'Coldest_Tmin']
print(f'ssp126+ssp370 Errornous variables: {
df[df.variable.str.contains('|'.join(errors), na=False, regex=True)].variable.unique()}\n')
if center in ['AMES', 'LARC', 'GISS', 'JPL', 'JSC', 'KSC', 'WFF']:
    # Remove rows with erroneous variables for specific scenarios
    df = df[~(df.variable.str.contains('|'.join(errors), na=False, regex=True) &
              df.scenario.isin(['ssp126', 'ssp370']))]
######################################

print(f'{len(files)} {center} files')
print(files[:5], '\n')
display(df.info())

ssp126+ssp370 Errornous variables: ['Coldest_Tmin_c' 'Max_DTR_c' 'tmin_frostdays_0C_d'
 'tmin_tropnights_20C_d']

45 LARC files
['updated_extremes/Coldest_Tmin_LARC_ssp126.csv', 'updated_extremes/Coldest_Tmin_LARC_ssp245.csv', 'updated_extremes/Coldest_Tmin_LARC_ssp370.csv', 'updated_extremes/HI_days_100F_LARC_ssp126.csv', 'updated_extremes/HI_days_100F_LARC_ssp245.csv'] 

<class 'pandas.core.frame.DataFrame'>
Index: 3330 entries, 90 to 3959
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   variable       3330 non-null   object 
 1   scenario       3330 non-null   object 
 2   term           3330 non-null   object 
 3   years          3330 non-null   float64
 4   ACCESS-CM2     3330 non-null   float64
 5   ACCESS-ESM1-5  3330 non-null   float64
 6   BCC-CSM2-MR    3330 non-null   float64
 7   CESM2          1620 non-null   float64
 8   CMCC-ESM2      3330 non-null   float64
 9   CNRM-CM6-1     3330 non-null   f

None

# Calculations

In [6]:
# Extract column names
mme = list(df.filter(regex='mme-').columns)
models = list(df.columns.drop(cols+mme+['years']))

# Calculate term-wise statistics
# For variables ending with '_d', use median
# For variables ending with '_c', use mean
term_mme = pd.concat([df[df.variable.str.endswith('_d')].groupby(cols)[mme].median(),
                      df[df.variable.str.endswith('_c')].groupby(cols)[mme].mean()
                     ]).reset_index().sort_values(cols, ascending=[1, 1, 0],
                                                  ignore_index=True)
display(term_mme.head())

,variable,scenario,term,mme-mean,mme-median,mme-pct25,mme-pct75
0,Coldest_Tmin_c,ssp245,short,-11.268358,-10.740965,-13.503494,-8.679038
1,Coldest_Tmin_c,ssp245,mid,-10.260196,-9.926980,-12.113423,-7.821688
2,Coldest_Tmin_c,ssp245,long,-8.900938,-8.415058,-10.516808,-6.578249
3,HI_days_100F_d,ssp126,short,13.250000,11.500000,5.250000,19.000000
4,HI_days_100F_d,ssp126,mid,16.159091,14.500000,7.625000,22.750000


In [7]:
# # Recalculate mme columns
# df['calc_mean'] = df[models].mean(axis=1)
# df['calc_median'] = df[models].median(axis=1)
# df['calc_pct25'] = df[models].quantile(0.25, axis=1)
# df['calc_pct75'] = df[models].quantile(0.75, axis=1)
# calc = ['calc_mean', 'calc_median', 'calc_pct25', 'calc_pct75']

# # Calculate term-wise statistics
# # For variables ending with '_d', use median
# # For variables ending with '_c', use mean
# term_calc = pd.concat([df[df.variable.str.endswith('_d')].groupby(cols)[calc].median(),
#                       df[df.variable.str.endswith('_c')].groupby(cols)[calc].mean()
#                      ]).reset_index().sort_values(cols, ascending=[1, 1, 0],
#                                                   ignore_index=True)
# # Rename columns to match term_mme
# term_calc.columns = term_mme.columns

# # Check if original mme and recalculated values are equal
# if min((term_mme.round(10) == term_calc.round(10)).sum()) != len(term_mme):
#     print(term_mme.compare(term_calc, result_names=('term_mme', 'term_calc')))
#     raise

# display(term_calc.head())

## Calculate Change Per (variable, scenario)
- short - mid
- short - long
- mid - long

In [8]:
query = """
SELECT
    a.variable,
    a.scenario,
    CASE
        WHEN a.term = 'short' AND b.term = 'mid' THEN 'short-mid'
        WHEN a.term = 'short' AND b.term = 'long' THEN 'short-long'
        WHEN a.term = 'mid' AND b.term = 'long' THEN 'mid-long'
    END AS term_diff,
    b.'mme-mean' - a.'mme-mean' AS 'mme-mean',
    b.'mme-median' - a.'mme-median' AS 'mme-median',
    b.'mme-pct25' - a.'mme-pct25' AS 'mme-pct25',
    b.'mme-pct75' - a.'mme-pct75' AS 'mme-pct75'
FROM term_mme a
JOIN term_mme b
    ON a.variable = b.variable
    AND a.scenario = b.scenario
    AND (
        (a.term = 'short' AND b.term = 'mid') OR
        (a.term = 'short' AND b.term = 'long') OR
        (a.term = 'mid' AND b.term = 'long')
    )
ORDER BY 1, 2, 3 DESC
"""

change = psql.sqldf(query, locals())

display(change.head(2))

,variable,scenario,term_diff,mme-mean,mme-median,mme-pct25,mme-pct75
0,Coldest_Tmin_c,ssp245,short-mid,1.008162,0.813985,1.390072,0.857351
1,Coldest_Tmin_c,ssp245,short-long,2.367420,2.325906,2.986686,2.100789


# Results
## Degree C

In [9]:
# Degrees C
agg_c = (term_mme[term_mme.variable.str.endswith('_c')]
         .groupby(['variable', 'term'])
         .agg({'mme-mean': ['min', 'max']})
         .sort_values(['variable', 'term'], ascending=[1, 0]))

agg_c['rounded_min'], agg_c['rounded_max'] = agg_c['mme-mean'].round(1).values.T
agg_c['diff'] = (agg_c[('mme-mean', 'max')] - agg_c[('mme-mean', 'min')]).round(1)

agg_c


mme-mean            rounded_min rounded_max diff
                            min        max                             
variable       term                                                    
Coldest_Tmin_c short -11.268358 -11.268358       -11.3       -11.3  0.0
               mid   -10.260196 -10.260196       -10.3       -10.3  0.0
               long   -8.900938  -8.900938        -8.9        -8.9  0.0
Hottest_Tmax_c short  39.078562  39.775092        39.1        39.8  0.7
               mid    39.553230  40.452730        39.6        40.5  0.9
               long   39.878945  41.974741        39.9        42.0  2.1
Max_DTR_c      short  24.678760  24.678760        24.7        24.7  0.0
               mid    24.469513  24.469513        24.5        24.5  0.0
               long   24.211326  24.211326        24.2        24.2  0.0

## Days

In [10]:
# Days
agg_d = (term_mme[term_mme.variable.str.endswith('_d')]
          .groupby(['variable', 'term'])
          .agg({'mme-median': ['min', 'max']})
          .sort_values(by=['variable', 'term'], ascending=[1, 0]))

round_up_half = lambda x: np.ceil(x) if x % 1 == 0.5 else round(x)

agg_d['rounded_min'] = agg_d[('mme-median', 'min')].apply(round_up_half).astype(int)
agg_d['rounded_max'] = agg_d[('mme-median', 'max')].apply(round_up_half).astype(int)
agg_d['diff'] = (agg_d[('mme-median', 'max')] - agg_d[('mme-median', 'min')]).astype(int)

agg_d

mme-median         rounded_min rounded_max diff
                                   min     max                             
variable              term                                                 
HI_days_100F_d        short       6.25   13.25           6          13    7
                      mid        12.50   20.75          13          21    8
                      long       14.75   44.25          15          44   29
prec_days_90th_d      short      21.00   21.25          21          21    0
                      mid        22.00   22.50          22          23    0
                      long       21.50   24.00          22          24    2
prec_days_95th_d      short      11.00   11.00          11          11    0
                      mid        11.50   12.00          12          12    0
                      long       11.50   13.00          12          13    1
prec_days_99th_d      short       2.00    2.50           2           3    0
                      mid         2.50    2.75           3           3    0
                      long        3.00    3.25           3           3    0
prec_days_dry_d       short     155.25  158.00         155         158    2
                      mid       156.00  156.75         156         157    0
                      long      154.00  154.75         154         155    0
prec_days_oneinch_d   short       6.00    6.00           6           6    0
                      mid         6.00    7.00           6           7    1
                      long        6.50    7.75           7           8    1
tmax_days_35C_d       short      15.00   16.50          15          17    1
                      mid        18.00   23.00          18          23    5
                      long       18.00   39.00          18          39   21
tmax_days_90th_d      short      50.50   52.00          51          52    1
                      mid        56.50   63.50          57          64    7
                      long       57.00   88.00          57          88   31
tmax_days_95th_d      short      28.00   30.00          28          30    2
                      mid        32.00   37.00          32          37    5
                      long       30.50   58.00          31          58   27
tmax_days_99th_d      short       5.50    7.50           6           8    2
                      mid         8.00   11.00           8          11    3
                      long        8.00   21.00           8          21   13
tmin_frostdays_0C_d   short      44.00   44.00          44          44    0
                      mid        39.00   39.00          39          39    0
                      long       34.00   34.00          34          34    0
tmin_tropnights_20C_d short      86.00   86.00          86          86    0
                      mid        94.00   94.00          94          94    0
                      long      102.00  102.00         102         102    0